In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

import matplotlib.pyplot as plt
from IPython.display import display
plt.style.use("fivethirtyeight")
sns.set_style({'font.sans-serif':['simhei','Arial']})
%matplotlib inline


# 检查Python版本
from sys import version_info
if version_info.major != 3:
    raise Exception('请使用Python 3 来完成此项目')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.cross_validation import cross_val_score

from scipy import stats
import seaborn as sns
from copy import deepcopy

%matplotlib inline

# This may raise an exception in earlier versions of Jupyter
%config InlineBackend.figure_format = 'retina'

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.set_option('display.max_rows',23)

In [4]:
train_data =pd.read_csv("d:/train.csv")
print(train_data.head())
test_data =pd.read_csv("d:/test.csv")
print(test_data.shape)
print(test_data.head())

   时间       小区名  小区房屋出租数量  楼层       总楼层      房屋面积     区     位置        月租金  \
0   1 -0.075191  0.128906   2  0.236364  0.008628  11.0  118.0   5.602716   
1   1 -0.035647  0.132812   1  0.381818  0.017046  10.0  100.0  16.977929   
2   1  1.162035  0.042969   0  0.290909  0.010593  12.0  130.0   8.998302   
3   1 -0.059868  0.085938   2  0.581818  0.019199   7.0   90.0   5.602716   
4   1  0.967777  0.214844   0  0.545455  0.010427   3.0   31.0   7.300509   

   房屋数量  房屋朝向_东  房屋朝向_东北  房屋朝向_东南  房屋朝向_北  房屋朝向_南  房屋朝向_西  房屋朝向_西北  房屋朝向_西南  \
0     3       0        0        1       0       0       0        0        0   
1     1       1        0        0       0       0       0        0        0   
2     5       0        0        1       0       0       0        0        0   
3     7       0        0        0       0       1       0        0        0   
4     4       0        1        0       0       0       0        0        0   

   房屋朝向_nan  
0         0  
1         0  
2         0  
3     

In [5]:
train_data['月租金']

0          5.602716
1         16.977929
2          8.998302
3          5.602716
4          7.300509
5          4.923599
6          6.621392
7         14.091681
8          4.584041
9         39.558574
10         4.923599
            ...    
196528     8.319185
196529     5.602716
196530     4.244482
196531     5.602716
196532     4.923599
196533     7.640068
196534    17.487267
196535     5.602716
196536     2.886248
196537     2.207131
196538    10.696095
Name: 月租金, Length: 196539, dtype: float64

In [6]:
df =train_data.copy()

In [7]:
# 特征变量和目标变量
features = df.drop('月租金', axis=1)
prices = df['月租金']

In [8]:
# 把分类特征都转成数值型后有{}行{}列
print('租房房价有数据 {0} 条，字段 {1} 个' .format(*df.shape))

# 导入 sklearn 进行训练测试集划分
from sklearn.model_selection import train_test_split

features_train, features_test, prices_train, prices_test = train_test_split(features, prices, test_size=0.3, random_state=0)

租房房价有数据 196539 条，字段 19 个


In [10]:
print(pd.isnull(features).values.any())
print(pd.isnull(prices).values.any())
features.describe()

False
False


,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,区,位置,房屋数量,房屋朝向_东,房屋朝向_东北,房屋朝向_东南,房屋朝向_北,房屋朝向_南,房屋朝向_西,房屋朝向_西北,房屋朝向_西南,房屋朝向_nan
count,196539.000000,1.965390e+05,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.000000,196539.0
mean,2.115229,1.292082e-15,0.124136,0.955449,0.408711,0.013139,7.904971,67.947685,4.760078,0.162624,0.016724,0.277355,0.053058,0.324897,0.049858,0.026351,0.089132,0.0
std,0.786980,1.000000e+00,0.133305,0.851511,0.183100,0.008104,4.025718,43.522332,1.636453,0.369023,0.128237,0.447694,0.224150,0.468338,0.217652,0.160177,0.284936,0.0
min,1.000000,-1.593672e+00,0.007812,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,1.000000,-9.075876e-01,0.039062,0.000000,0.290909,0.009268,4.000000,33.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,2.000000,-6.827065e-02,0.082031,1.000000,0.418182,0.012910,9.000000,61.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,3.000000,9.761797e-01,0.160156,2.000000,0.563636,0.014896,11.000000,103.000000,6.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0
max,3.000000,1.682036e+00,1.000000,2.000000,1.000000,1.000000,14.000000,152.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [11]:
# 建立模型
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [12]:
# from sklearn.metrics import r2_score
def performance_metric(y_true, y_predict):
    score =mean_squared_error(y_true, y_predict)**0.5
    return score

In [13]:
# 通过交叉认证缓解数据集过拟合的现象
# 建立决策树回归模型
# 通过GridSearchCV找到最优深度参数（基于输入数据[X,y] 利于网格搜索找到最优的决策树模型）
def fit_model(X, y):
    
    cross_validator = KFold(10, shuffle=True)
    regressor = DecisionTreeRegressor()
    params = {'max_depth': [1,2,3,4,5,6,7,8,9,10]}
    scoring_fnc = make_scorer(performance_metric)
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cross_validator)
    
    # 网格搜索
    grid = grid.fit(X, y)
    return grid.best_estimator_

In [14]:
optimal = fit_model(features_train, prices_train)

In [40]:
print('最优模型的参数 max_depth 是: {} ' .format(optimal.get_params()['max_depth']))

最优模型的参数 max_depth 是: 1 


In [15]:
optimal.get_params()

{'criterion': 'mse',
 'max_depth': 1,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [16]:
predicted_value = optimal.predict(features_test)
rmse = performance_metric(prices_test, predicted_value)

# 每次交叉验证得到的数据集不同，因此每次运行的结果也不一定相同
print('最优模型在测试数据上 rmse 分数 {: .2f}' .format(rmse))

最优模型在测试数据上 rmse 分数  5.32


In [19]:
from sklearn.ensemble import RandomForestRegressor

np.random.seed(21)

rf_reg = RandomForestRegressor(n_estimators=50, verbose=1)
rf_reg.fit(features_train, prices_train)

combine_lists = lambda item: [item[0], item[1]]
feature_importance = list(map(combine_lists, zip(features_train.columns, rf_reg.feature_importances_)))
feature_importance = pd.DataFrame(
    feature_importance, columns=['feature', 'importance']
).sort_values(by='importance', ascending=False)

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   29.5s finished


In [20]:
feature_importance.head(23)

,feature,importance
5,房屋面积,0.482425
6,区,0.115762
4,总楼层,0.105577
1,小区名,0.083544
7,位置,0.072956
2,小区房屋出租数量,0.051483
8,房屋数量,0.038045
3,楼层,0.016052
13,房屋朝向_南,0.006114
0,时间,0.005881


In [23]:
# filter_feature = feature_importance[feature_importance['importance'] > 0.001]['feature'].tolist()
# filter_feature

In [22]:
filter_feature = feature_importance['feature'].tolist()
filter_feature

['房屋面积',
 '区',
 '总楼层',
 '小区名',
 '位置',
 '小区房屋出租数量',
 '房屋数量',
 '楼层',
 '房屋朝向_南',
 '时间',
 '房屋朝向_东',
 '房屋朝向_东南',
 '房屋朝向_西南',
 '房屋朝向_西',
 '房屋朝向_北',
 '房屋朝向_西北',
 '房屋朝向_东北',
 '房屋朝向_nan']

In [24]:
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [25]:
def other_regression_model(reg, train_data, train_y, val_data, val_y, input_feature):
    reg.fit(train_data.as_matrix(columns=input_feature), train_y)
    RMSE = mean_squared_error(
        val_y, reg.predict(val_data.as_matrix(columns=input_feature))
    ) ** 0.5
    return reg, RMSE

In [26]:
# features_train2, features_test2, prices_train2, prices_test2
ridge = Ridge()
dt_gre = DecisionTreeRegressor()
rf_reg = RandomForestRegressor()
ridge, RMSE = other_regression_model(ridge, features_train, prices_train, features_test, prices_test, filter_feature)
print('RMSE for ridge_regression is %s' % RMSE)
dt_gre, RMSE = other_regression_model(dt_gre, features_train, prices_train, features_test, prices_test, filter_feature)
print('RMSE for DecisionTreeRegressor is %s' % RMSE)
rf_reg, RMSE = other_regression_model(rf_reg, features_train, prices_train, features_test, prices_test, filter_feature)
print('RMSE for RandomForestRegressor is %s' % RMSE)

RMSE for ridge_regression is 5.246094322861433
RMSE for DecisionTreeRegressor is 1.8395359421094897
RMSE for RandomForestRegressor is 1.6377819750062503


In [27]:
from sklearn.externals import joblib
import os

In [28]:
os.chdir("d:/model_save/")
joblib.dump(rf_reg, "RandomForestRegressor.m")


['RandomForestRegressor.m']

In [29]:
new_rf_reg = joblib.load("RandomForestRegressor.m")

In [30]:
df1 = test_data.copy()

In [34]:
# 特征变量和目标变量
features_test1 = df1.drop('id', axis=1)
print(features_test1.head())
id_test1 = df1['id']
print(id_test1.head())

   时间       小区名  小区房屋出租数量  楼层       总楼层      房屋面积     区    位置  房屋数量  房屋朝向_东  \
0   4  1.362147  0.382812   1  0.600000  0.007117  10.0   5.0     4       1   
1   4 -0.750335  0.152344   1  0.472727  0.007448   3.0   0.0     4       1   
2   4 -1.212104  0.207031   2  0.709091  0.014068  10.0   9.0     7       0   
3   4 -0.952084  0.011719   0  0.090909  0.008937   6.0  96.0     4       0   
4   4  0.859245  0.035156   1  0.218182  0.008606   6.0  61.0     4       0   

   房屋朝向_东北  房屋朝向_东南  房屋朝向_北  房屋朝向_南  房屋朝向_西  房屋朝向_西北  房屋朝向_西南  房屋朝向_nan  
0        0        0       0       0       0        0        0         0  
1        0        0       0       0       0        0        0         0  
2        0        1       0       0       0        0        0         0  
3        0        0       0       1       0        0        0         0  
4        0        1       0       0       0        0        0         0  
0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64


In [35]:
tesr_result =new_rf_reg.predict(df1.as_matrix(columns=filter_feature))

In [39]:
price_pre = pd.DataFrame(tesr_result,index=list(range(0,len(tesr_result))),columns=['price'])
price_pre.head()

,price
0,4.012450
1,5.874363
2,12.563667
3,5.602716
4,4.991511


In [40]:
res01 = pd.concat([id_test1,price_pre],axis=1)
res01.head()

,id,price
0,1,4.012450
1,2,5.874363
2,3,12.563667
3,4,5.602716
4,5,4.991511


In [41]:
res01.to_csv('d:/res01.csv',index=False,header=True)

In [42]:
len(res01)

56279